# Exercise Chapitre 1
## SQLITE

#### 1) Read all the data from bc.csv, how many rows is there ? Columns ?

In [2]:
import pandas as pd

df = pd.read_csv('C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/bc.csv')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


**There are 569 rows and 33 columns**

#### 2) Insert the bc.csv in the sqlite format.

In [ ]:
import sqlite3

#  Establishes a connection to an SQLite database file. The only argument is the path (If the file does not exist and empty file is created)
conn = sqlite3.connect('C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/bc.db')

# A cursor in SQL databases is a database object that allows you to retrieve and manipulate a set of data rows returned by a SQL statement, one row at a time.
c = conn.cursor()

c.execute("""SELECT * FROM bc""")

docs = c.fetchall()


#### 3) In the same db create a new table called tomatch with two variables id_ and rdm_float.

In [5]:
conn = sqlite3.connect('C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/bc.db')
c = conn.cursor()

def create_table_tomatch():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS tomatch(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_tomatch()

#### 4) In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.

In [ ]:
import tqdm
import random

query = """INSERT INTO tomatch("id_","rdm_float") VALUES (?,?)"""

for i in tqdm.tqdm(range(50000,10000001,2)):
    random_float = random.random()
    values = (i,random_float)
    c.execute(query, values)
conn.commit()

c.execute("""SELECT * FROM tomatch""")
docs = c.fetchall()
print(docs[:10])

#### 5) Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

In [7]:
c.execute("""SELECT bc.id as id1, tomatch.id_ as id2
             FROM bc
             INNER JOIN tomatch
             ON bc.id = tomatch.id_
             """)
c.execute("""SELECT "radius_mean", diagnosis, AVG("radius_mean"), SUM("radius_mean"), COUNT("radius_mean"), MIN("radius_mean"), MAX("radius_mean")
             FROM bc 
             GROUP BY diagnosis
             """)

docs = c.fetchall()
print(docs)

[(17.85, 'B', 12.14652380952381, 4336.309, 357, 6.981, 17.85), (28.11, 'M', 17.462830188679245, 3702.12, 212, 10.95, 28.11)]


#### 6) How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

In [8]:
c.execute(""" SELECT radius_mean FROM bc where radius_mean > 15""")
print("Number of obs with radius_mean greater than 15 :",len(c.fetchall()))

Number of obs with radius_mean greater than 15 : 173


In [9]:
c.execute(""" SELECT texture_mean FROM bc where radius_mean > 15 AND texture_mean > 20""")
print("Number of obs with a texture_mean greater than 20, given that radius_mean is greater than 15:",len(c.fetchall()))

Number of obs with a texture_mean greater than 20, given that radius_mean is greater than 15: 106


#### 7) For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups

In [23]:
c.execute("""SELECT round(AVG("radius_mean"),2)
             FROM bc 
             GROUP BY diagnosis
             """)
res = c.fetchall()
print(res)

a = res[0]
b = res[1]

diff = tuple(x-y for x,y in zip(b,a))
print('Difference = ',diff)


[(12.15,), (17.46,)]
Difference =  (5.3100000000000005,)


#### 8) Change the name of the column "diagnosis" to "label"

In [25]:
c.execute("""ALTER TABLE bc RENAME COLUMN diagnosis TO label""") 

#### 9) Add a column named "day" of type DATE. 

In [ ]:
c.execute("""ALTER TABLE bc ADD COLUMN day DATE""")

#### 10) Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)

In [ ]:
c.execute("""ALTER TABLE bc DROP COLUMN area_mean1""")

In [35]:
c.execute("""ALTER TABLE bc RENAME COLUMN area_mean TO area_mean1""") 
c.execute("""ALTER TABLE bc ADD COLUMN area_mean INT""")
c.execute(""" UPDATE bc SET 'area_mean' = 'area_mean1'""")
c.execute("""ALTER TABLE bc DROP COLUMN area_mean1""")

print(c.fetchall())

[]


#### 11) create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.

In [ ]:
def create_table_tomatch2():
    try:
        c.execute("""CREATE TABLE IF NOT EXISTS tomatch2(id_ INT PRIMARY KEY, "rdm_float" FLOAT)""")
        conn.commit()
    except Exception as e:
        print(str(e))
        
# Sqlite3 excepts a table named "main" and therefore gives you a warning (can be ignored)
create_table_tomatch2()

query = """INSERT INTO tomatch2("id_","rdm_float") VALUES (?,?)"""

for i in tqdm.tqdm(range(50000,10000001,10)):
    random_float = random.random()
    values = (i,random_float)
    c.execute(query, values)
conn.commit()

c.execute("""SELECT * FROM tomatch2""")
docs = c.fetchall()
print(docs[:10])

#### 12) Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2.

In [3]:
c.execute("""SELECT bc.id as id1, tomatch.id_ as id2, tomatch2.id_ as id3
             FROM bc
             INNER JOIN tomatch ON bc.id = tomatch.id_
             INNER JOIN tomatch2 ON bc.id = tomatch2.id_
             WHERE radius_mean > 15 AND tomatch.rdm_float > 0.5 AND tomatch2.rdm_float > 0.5 
             """)

print(c.fetchall())

[(879830, 879830, 879830), (884180, 884180, 884180), (8911670, 8911670, 8911670), (8912280, 8912280, 8912280), (895100, 895100, 895100), (9012000, 9012000, 9012000)]


## MYSQL

In [4]:
!pip install mysql-connector-python

In [7]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root")

print(mydb) 

c = mydb.cursor()

c.execute("CREATE DATABASE m1ds2e")

In [8]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="m1_ds2e"
)

# Create a cursor object
c = conn.cursor()

# Execute the SQL query
c.execute("SELECT * FROM sample_superstore")

docs = c.fetchall()

columns = [description[0] for description in c.description]
df = pd.DataFrame(docs, columns=columns)

df.head(100)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,5/28/2012,5/30/2012,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/7/2010,7/8/2010,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,7/27/2011,7/27/2011,998.2023,8,1446.67,88523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25225,Not Specified,0.03,6.68,6.15,36,Ray Morgan,Regular Air,Small Business,Office Supplies,...,West,Oregon,Hillsboro,97123,11/27/2012,11/29/2012,5.6500,10,73.81,91256
96,25226,Not Specified,0.10,5.78,5.67,37,Stacy Chandler,Regular Air,Small Business,Office Supplies,...,West,Oregon,Keizer,97303,11/27/2012,11/30/2012,7.6700,11,62.59,91256
97,20935,Medium,0.02,1.68,1.57,39,Ian L O'Donnell,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Alton,62002,7/3/2011,7/4/2011,-30.5400,10,17.12,88682
98,20936,Medium,0.10,7.08,2.35,39,Ian L O'Donnell,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Alton,62002,7/3/2011,7/5/2011,23.8200,14,90.80,88682


In [10]:
df.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

In [13]:
df["Order Date"].tail(10)

165     3/2/2010
166     3/2/2010
167     3/2/2013
168    8/26/2013
169     5/2/2012
170     5/2/2012
171    6/12/2013
172     1/2/2010
173     1/2/2013
174     1/2/2013
Name: Order Date, dtype: object

#### 1: How many orders were placed in total?

In [14]:
c.execute("SELECT COUNT(*) FROM sample_superstore")
print("Number of orders:",c.fetchall())

Number of orders: [(175,)]


#### 2: What is the total sales amount?

In [15]:
c.execute("SELECT ROUND(SUM(sales),2) FROM sample_superstore")

In [16]:
print("Total amount of sales :",c.fetchall())

Total amount of sales : [(130555.31,)]


#### 3: What is the average profit per order?

In [17]:
c.execute("SELECT AVG(profit) FROM sample_superstore")
res = c.fetchall()
print("Average profit per order:",res[0])

Average profit per order: (136.04226416285724,)


#### 4: Which customer has placed the most orders?

In [18]:
c.execute("SELECT `Customer Name`, COUNT(*) AS obs FROM sample_superstore GROUP BY `Customer Name` ORDER BY obs DESC LIMIT 10")
results = c.fetchall()
for i in results:
    print(i[0], i[1])

Tony Wilkins Winters 12
Terri Lyons 8
Randall Montgomery 8
Bonnie Potter 6
Timothy Reese 6
Laurie Hanna 6
Charlotte Stanton 6
Guy Gallagher 5
Geoffrey Hewitt 5
Gene Diaz 5


#### 5: What is the total profit for each product category?

In [19]:
c.execute("SELECT `Product Category`, SUM(profit) AS total_profit FROM sample_superstore GROUP BY `Product Category`")
results = c.fetchall()
for row in results:
    print("Category:", row[0], "- Total Profit:", row[1])

Category: Office Supplies - Total Profit: 11605.983540000001
Category: Technology - Total Profit: 10225.340091999999
Category: Furniture - Total Profit: 1976.0725965000036


#### 6: What is the average discount percentage?

In [20]:
c.execute("SELECT AVG(Discount) FROM sample_superstore")
res = c.fetchall()
avg = round(res[0][0]*100,2)
print("Average discount:",avg)

Average discount: 4.97


#### 7: Which region has the highest total sales?

In [21]:
c.execute("SELECT `Region`, SUM(Sales) AS total_sales FROM sample_superstore GROUP BY `Region`ORDER BY total_sales DESC")
results = c.fetchall()
df = pd.DataFrame(results, columns=["Region","Total Sales"])
df

,Region,Total Sales
0,West,75678.50
1,East,38033.85
2,Central,14261.62
3,South,2581.34


In [ ]:
The West Region hast the highest total sales.

#### 8: How many orders were shipped using each shipping mode?

In [22]:
c.execute("SELECT `Ship Mode`, COUNT(*) AS obs FROM sample_superstore GROUP BY `Ship Mode`")
results = c.fetchall()
df = pd.DataFrame(results, columns=["Shipping mode","Total "])
df

,Shipping mode,Total
0,Regular Air,131
1,Express Air,23
2,Delivery Truck,21


#### 9: What is the total quantity ordered for each product sub-category?

In [23]:
c.execute("SELECT `Product Sub-Category`, SUM(`Quantity ordered new`) AS total_quantity FROM sample_superstore GROUP BY `Product Sub-Category`")
results = c.fetchall()
df = pd.DataFrame(results, columns=["Sub-Category","Total quantity"])
df

,Sub-Category,Total quantity
0,Labels,171
1,Pens & Art Supplies,196
2,Paper,286
3,"Scissors, Rulers and Trimmers",24
4,Telephones and Communication,252
5,Office Machines,123
6,Chairs & Chairmats,166
7,Appliances,184
8,Bookcases,39
9,Tables,50


#### 10: Which product has the highest profit margin?

In [24]:
c.execute("SELECT `Product Name`, MAX(`Product Base Margin`) as profit_margin FROM sample_superstore GROUP BY `Product Name` ORDER BY profit_margin DESC")
results = c.fetchall()
df = pd.DataFrame(results, columns=["Product Name","Profit Margin"])
df

,Product Name,Profit Margin
0,Accessory29,0.83
1,"IrisÂ® 3-Drawer Stacking Bin, Black",0.83
2,Accessory37,0.81
3,Space Solutions Commercial Steel Shelving,0.80
4,"Eldon Base for stackable storage shelf, platinum",0.80
...,...,...
144,Xerox 1903,0.36
145,"TOPS Voice Message Log Book, Flash Format",0.36
146,Xerox 193,0.36
147,Storex DuraTech Recycled Plastic Frosted Binders,0.35


It looks like 'Accessory29' and 'IrisÂ® 3-Drawer Stacking Bin, Black' have the highest profit margin of 0.83

#### 11: What is the total sales amount for each year?

#### 12: Which customer segment has the highest average sales?

In [4]:
c.execute("SELECT `Customer Segment`, AVG(Sales) AS avg_sales FROM sample_superstore GROUP BY `Customer Segment` ORDER BY avg_sales DESC")
results = c.fetchall()
df = pd.DataFrame(results, columns = ["Customer Segment", "Average Sales"])
df

,Customer Segment,Average Sales
0,Small Business,1041.564808
1,Home Office,868.696471
2,Consumer,856.527059
3,Corporate,528.821798


In [21]:
c.execute("""SELECT `Customer Segment`, AVG(Sales) as avg_sales, RANK() OVER (PARTITION BY `Customer Segment` ORDER BY AVG(Sales)) FROM sample_superstore GROUP BY `Customer Segment`""")

results = c.fetchall()
df = pd.DataFrame(results, columns = ["Customer Segment", "Average Sales","Rank"])
df

,Customer Segment,Average Sales,Rank
0,Consumer,856.527059,1
1,Corporate,528.821798,1
2,Home Office,868.696471,1
3,Small Business,1041.564808,1


## PostgreSQL

In [25]:
df = pd.read_csv('C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/bank_full.csv')
df.columns
df

,age,job,marital,education,default_,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [22]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
    --------------------------------------- 0.0/1.2 MB 320.0 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.2 MB 544.7 kB/s eta 0:00:03
   -------- ------------------------------- 0.2/1.2 MB 1.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 0.9/1.2 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.5 MB/s eta 0:00:00


In [1]:
import psycopg2
import pandas as pd


# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="m1ds2e", user="postgres", password="root", host="localhost",  port="5432" )
cur = conn.cursor()

#### 1: What is the average age of the customers

In [35]:
table_name = "bank_full"
columns = [
    "age INTEGER",
    "job VARCHAR(100)",
    "marital VARCHAR(20)",
    "education VARCHAR(100)",
    "default_ VARCHAR(10)",
    "balance NUMERIC",
    "housing VARCHAR(10)",
    "loan VARCHAR(10)",
    "contact VARCHAR(100)",
    "day INTEGER",
    "month VARCHAR(20)",
    "duration INTEGER",
    "campaign INTEGER",
    "pdays INTEGER",
    "previous INTEGER",
    "poutcome VARCHAR(100)",
    "y VARCHAR(10)"
]

create_table_query = (
    "CREATE TABLE IF NOT EXISTS " + table_name + " (" +
    ", ".join(columns) +
    ")"
)

cur.execute(create_table_query)
conn.commit()

#### 1: What is the average age of the customers?

In [11]:
cur.execute("SELECT AVG(age) FROM bank_full")
res = cur.fetchall()
print("Average age of customers:",int(float(res[0][0])))

Average age of customers: 40


#### 2: How many customers have defaulted on loans?

In [6]:
cur.execute("SELECT default_, COUNT(*) AS obs FROM bank_full GROUP BY default_")
print("Number of customers who have defaulted on loans:",cur.fetchall()[1][1])

Number of customers who have defaulted on loans: 815


#### 3: What is the maximum balance in the account?

In [8]:
cur.execute("SELECT MAX(balance) FROM bank_full")
res = cur.fetchall()
print("Maximum balance in account:",int(float(res[0][0])),"€") #In Euro since it is portugese information

Maximum balance in account: 102127 €


#### 4: How many customers have a housing loan?

In [23]:
cur.execute("SELECT housing, COUNT(*) AS obs FROM bank_full GROUP BY housing")
print("Number of customers with housing loans:",cur.fetchall()[1][1])

Number of customers with housing loans: 25130


#### 5: What is the education level of the youngest customer?

In [11]:
import pandas as pd

cur.execute("SELECT age, COUNT(*) as obs FROM bank_full GROUP BY age ORDER BY age")
res = cur.fetchall()
df = pd.DataFrame(res, columns = ["Age", "Frequency"])
df

,Age,Frequency
0,18,12
1,19,35
2,20,50
3,21,79
4,22,129
...,...,...
72,90,2
73,92,2
74,93,2
75,94,1


In [ ]:
cur.execute("SELECT education, MIN(age) AS min_age FROM bank_full GROUP BY age")
print(cur.fetchall())

In [ ]:
cur.execute("SELECT MIN(age) FROM bank_full")
print(cur.fetchall())

In [9]:
cur.execute("SELECT education FROM bank_full WHERE age = (SELECT MIN(age) FROM bank_full)")
print(cur.fetchall())

[('primary',), ('unknown',), ('primary',), ('unknown',), ('secondary',), ('unknown',), ('unknown',), ('secondary',), ('primary',), ('unknown',), ('unknown',), ('unknown',)]


#### 6: What is the marital status of the oldest customer?

Just like before, we see from the table above that two customers share the oldest age in our data. We can see the marital status of both

In [3]:
cur.execute("SELECT marital FROM bank_full WHERE age = (SELECT MAX(age) FROM bank_full)")
print(cur.fetchall())

[('divorced',), ('married',)]


#### 7: What is the distribution of the months in which customers were last contacted?

In [ ]:
import pandas as pd

cur.execute("SELECT month, COUNT(*) as obs FROM bank_full GROUP BY month ORDER BY obs DESC")
res= cur.fetchall()
df = pd.DataFrame(res, columns = ["Month","Frequency"])
df

We can see that may is the month in which most customers were last contacted

#### 8: What is the average duration of contact for customers who have a personal loan?

In [7]:
cur.execute("SELECT AVG(CASE WHEN loan = 'yes' THEN duration ELSE NULL END) AS avg_duration_with_loan FROM bank_full")
res = cur.fetchall()
print("Average duration of contact for customers who have a personal loan:\n",int(float(res[0][0])),"seconds")

Average duration of contact for customers who have a personal loan:
 250 seconds


#### 9: How many customers were previously contacted, and what was the average duration of their last contact?

In [6]:
cur.execute("SELECT COUNT(previous) FROM bank_full WHERE previous > 0")
prev = cur.fetchall()

cur.execute("SELECT AVG(duration) FROM bank_full WHERE previous > 0")
avg = cur.fetchall()

print("Number of customers previously contacted:", int(float(prev[0][0])), "\nAverage duration of their last contact:",int(float(avg[0][0])),"seconds")

Number of customers previously contacted: 8257 
Average duration of their last contact: 260 seconds


#### 10: What is the distribution of job types among customers?

In [24]:
cur.execute("SELECT job, COUNT(*) AS obs FROM bank_full GROUP BY job ORDER BY obs DESC")
res= cur.fetchall()
df = pd.DataFrame(res, columns = ["Job Type","Frequency"])
df

,Job Type,Frequency
0,blue-collar,9732
1,management,9458
2,technician,7597
3,admin.,5171
4,services,4154
5,retired,2264
6,self-employed,1579
7,entrepreneur,1487
8,unemployed,1303
9,housemaid,1240


#### 11: How many customers have been contacted more than once in the current campaign?

In [13]:
cur.execute("SELECT COUNT(campaign) FROM bank_full WHERE campaign > 1")
res = cur.fetchall()

print("Number of customers contacted more than once in the current campaign:", int(float(res[0][0])))

Number of customers contacted more than once in the current campaign: 27667


#### 12: What is the success rate of previous marketing campaigns (poutcome) among customers

Customers who haven't been contacted during the previous campaign have a poutcome of 'unkown', and thus shouldn't be taken into consideration when calculating the ratio

In [ ]:
cur.execute("SELECT poutcome, COUNT(*) AS obs FROM bank_full GROUP BY poutcome") gf 
res = cur.fetchall()

other = res[0][1]
failure = res[1][1]
success = res[3][1]

success_rate = int(success/(other+failure+success)*100)
print("Success rate:",success_rate,"%")